<a href="https://colab.research.google.com/github/dmkant/Camino/blob/master/Rendu_Intermediaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center><b>Compte-Rendu Intermediaire</b></center></h1>

## Bilan
Je me présente je m'appelle Henri. Je voudrai bien réussir ma vi être aiméééé !

## Formalisation du problème

Soit G = (V, A) un graphe orienté avec : 

*   |V | = n
*   |A| = m
	
Soit R un ensemble de ressources avec |R| = $n_r$.
A chaque arc (i,j) de A, on associe un cout c<sub>i,j</sub>  (on suppose positif ) et un vecteur de consommation de ressource t<sub>i,j</sub>   de dimension $n_r$  
On considère des contraintes de ressources finales portant uniquement sur la borne supérieure.

**Objectif:**
On cherche a trouver un chemin qui minimise le coût et dont la consommation de ressource (noté T) respecte les contraintes.









## Modélisation et génération du graph
Nous avons décidé de modéliser nos graphs à l'aide de la POO. Vous pouvez voir ci-dessous la classe *Graph* dont chaque instance contient un attribut *ressource* (liste contenant l'ensemble des ressources) et un attribut *graph_dict* (dictionnaire contenant l'ensemble des arcs avec leurs coût et leur consommation de ressource)


In [0]:
import numpy as np

"""
    Modélisation d'un graphe : classe graphe
    Le graphe sera modélisé sous la forme d'un double-dictionnaire 
    arc[S1 : sommet origine][S2 : sommet but ] = vecteur 
    Si le vecteur existe, alors il y a un arc entre S1 et S2
    Le vecteur sera de taille 1+R et représentera le vecteur cout et ressources de l'arc
    vecteur[0] = cout de l'arc
    vecteur[i] = Consommation ressource i sur l'arc ou i = [| 1 ; R |]
    
"""
class Graphe:
    nb_graph=0

    #res = nom des ressources à considérer
    def __init__(self,res=None):
        self.arc={} #double dictionnaire 
        self.noeud=[] #noeud
        if(type(res) is not list):
          res=[res]
        self.ressource=res # nom des ressources
        #Nombre de ressource
        if(res==[None]):
          self.nb_ressources = 0
        else:  
          self.nb_ressources = len(res) 
        Graphe.nb_graph+=1
    
    #Fonction permettant d'ajouter une/des ressources
    def ajout_ressource(self,nom_ress):
      if (type(nom_ress) is not list):
        nom_ress=[nom_ress]
      for nom in nom_ress:
        if(self.ressource==[None]):
          self.ressource=[nom]
        else: 
          self.ressource.append(nom)
        self.nb_ressources+=1
        for source in self.arc.keys():
          for cible in self.arc[source].keys():
           self.arc[source][cible]=np.append(self.arc[source][cible],0)
        
    #Fonction permettant l'ajout d'un/des sommet au graphe
    def ajoutNoeud(self,sommet):
      if (type(sommet) is not list):
        sommet=[sommet]
      for som in sommet:
        if (som in self.noeud):
          print('Le sommet ',som,' est déjà présent dans votre graph')
        else:
          self.noeud.append(som)
    
    #Fonction permettant l'ajout d'un/des arc entre 2 sommets
    def ajoutArc(self,origin,dest,conso):
        #conso = Cout de l'emprunt de l'arc et consommation des ressources
        if(type(origin) is not list):
          origin=[origin]
        if(type(dest) is not list):
          dest=[dest]
        if (type(conso) is not list):
          conso=[conso]
        if(type(conso[0]) is not list):
          conso=[conso]
        for i in range(len(origin)):
          if(len(conso[i])==self.nb_ressources+1):
            if(origin[i] not in self.noeud):
                self.ajoutNoeud(origin[i])
            if(origin[i] not in self.arc.keys()):
                self.arc[origin[i]]={}
            if (dest[i] not in self.noeud):
              self.ajoutNoeud(dest[i])
            self.arc[origin[i]][dest[i]]=np.array(conso[i])
          else:
            print("Le vecteur conso de l'arc (",origin[i],dest[i],") n'a pas la bonne dimention. L'arc n'a donc pas été ajouté") 
        
    #Fonction retournant les predecesseurs d'un sommet sous forme de liste    
    def predecesseurs(self,sommet):
        liste_pred = []
        for key in self.arc.keys():
            if(sommet in self.arc[key].keys()):
                liste_pred.append(key)  
        return(liste_pred)
        
    #Fonction retournant les successeurs d'un sommet sous forme de liste
    def successeurs(self,sommet):
        liste_suc = []
        for key in self.arc[sommet].keys():
            liste_suc.append(key)
        return(liste_suc)
    
    #Fonction affichant le graphe sous forme
    # Sommet(nombre de successeurs) : [{successeur_i : vecteur conso }, ... ]
    def affiche(self):
        for key in self.arc.keys():
            print(key+"("+str(len(self.arc[key]))+"): [ ",end="")
            for cle in self.arc[key].keys():
                print("{"+cle+" : "+str(self.arc[key][cle])+" } ,",end="")
            print("]")


In [0]:
#Exemple d'utilisation
g=Graphe()
g.ajout_ressource("fatigue")
g.ajout_ressource(["prix essence","cris des enfants"])
g.ajoutNoeud("Paris")
g.ajoutNoeud(["Berlin","Tokyo"])
g.ajoutArc("Paris","Berlin",[1,2,3,4])
g.ajoutArc(["Berlin","Tokyo"],["Tokyo","Tokyo"],[[1,2,3,4],[4,3,2,1]])
print(g.ressource)
print(g.noeud)
print(g.arc)
g.affiche()

['fatigue', 'prix essence', 'cris des enfants']
['Paris', 'Berlin', 'Tokyo']
{'Paris': {'Berlin': array([1, 2, 3, 4])}, 'Berlin': {'Tokyo': array([1, 2, 3, 4])}, 'Tokyo': {'Tokyo': array([4, 3, 2, 1])}}
Paris(1): [ {Berlin : [1 2 3 4] } ,]
Berlin(1): [ {Tokyo : [1 2 3 4] } ,]
Tokyo(1): [ {Tokyo : [4 3 2 1] } ,]


## Résolution d'un problème de Plus Court Chemin avec Contraintes(PCCC)
#### **Definition**

* **Etiquette** : vecteur associé à un chemin et dont les coordonnées sont le coût et les consommations des différentes ressources.
* **Orde de Pareto** : Une étiquette E domine E' ssi toute les cordonnées de E sont supérieur ou égale à celle de E' et qu’au moins une coordonnée de E est strictement supérieur à celle de E'

#### **Algorithme à correction d'étiquettes**
On utilisera **l'algorithme à correction d'étiquettes**. 
On considère un graphe de source le sommet s et de puit le sommet t.
A chaque sommet i on associe un ensemble d'étiquettes E<sub>s,i</sub> qui correspondes à des chemins entre la source et ce sommet.
A l'initialisation, tous les ensembles E<sub>s,i</sub> sont vides car aucun chemin n'a été traité exépté E<sub>s,s</sub> qui vaut {0}.


On condisere une liste *L* qui contient l'ensemble des sommet dont au moins une étiquette n'a pas encore été propagé, et qu'on met a jour a chaque itération.
  
A chaque intération, on choisi un sommet de *L* et on propage ses étiquette à ses successeurs si cette propagation respecte les contraintes. On élimine ce sommet de *L*. On ajoute ainsi de nouvelles étiquettes à ses successeur sur lesquelles on effectuera un test de domination au sens de l'ordre de Pareto afin de ne conserver que les étiquettes non-dominé. Ce test de domination permet d'éventuellement **corriger** les étiquettes déjà existantes , d'où le nom de l'algorithme.

A chaque fois qu'on crée une nouvelle étiquette qui est maintenu après le test de domination, le sommet qui lui correspond est ajouté à la liste *L*. L'algorithme s'arrête une fois que la liste est vide donc lorsque tous les sommets sont traités et que les propagations ne change plus les étiquettes.

Sur tous les sommets et particulièrement en t, l'algorithme permet d'obtenir une ou plusieurs étiquettes non dominées  donc candidates comme solutions du problème.
Etant donné que le but est de minimiser le cout de s à t, l'algorithme renvoie le chemin ayant le cout le plus faible en t. S'il y en a plusieurs, on en choisit un. 
Notons que dans cet algorithme, on calcule le plus court chemin contraint de s à tous les autres sommets du graphe.

![Algorithme](https://drive.google.com/uc?id=1GNPnNF0-vO_jykku-rCmQeSDKuthGv9E)


In [0]:
""" Création d'une fonction Pareto qui prend comme paramètre un ensemble d'étiquette 
(et argument facultatif correspondant aux prédécésseur de ces etiquette) et renvoie
un ensemble Pareto minimale
"""
    
def ordre_Pareto(arc1,arc2):
    arc = arc1 - arc2
    retour = None
    if np.all(arc>=0)==True:
        retour =np.array([arc2])
    elif np.all(arc<0)==True:
        retour = np.array([arc1.tolist()])
    else:
        retour = np.array([arc1,arc2])
    return(retour)

#Fonction qui permet de supprimer les doublons d'un ensemble d'étiquette avec leur prédécesseurs
def supp_doublon(E,P):
  #enleve les valeurs dupliques
  uniq_value=np.unique(E,axis=0,return_counts=True)
  if(P is not None):
    doublons=uniq_value[0][uniq_value[1]>1]
    for d in doublons:
      succ_doub_ind=np.array([i for i, e in enumerate(E) if np.all(e==d)])
      #Selectionne une etiquette aléatoirement correspondant au doublon d
      choix=np.random.choice(succ_doub_ind)
      #Supprimer les autres etiquette
      suppression=succ_doub_ind[np.array(succ_doub_ind!=choix)]
      conserver=np.array([i for i in range(len(P))])
      conserver=conserver[[i for i in range(len(P)) if conserver[i] not in suppression]]
      P=P[conserver]
  E=uniq_value[0]
  return([E,P])

#Fonction renvoie l'ensemble Pareto optimal d'un ensemble d'étiquette avec leurs prédécesseur 
def Pareto(Etiq,Pred=None):
   #Enleve les étiquettes dupliqués
   [Etiq,Pred]=supp_doublon(Etiq,Pred)
   #Conserve uniquement les element non dominés
   i=0
   while(i < len(Etiq)): 
     element1=Etiq[i]
     j=0
     while(j < len(Etiq)):
       if(i!=j):
        element2=Etiq[j]
        test=ordre_Pareto(element1,element2)
        if(len(test)==1):
          supp=j if np.all(element1==test[0]) else i
          Etiq=np.delete(Etiq,supp,axis=0)
          if(Pred is not None):
            Pred=np.delete(Pred,supp)
       j+=1 
     i+=1
   resu=[Pred,Etiq] if Pred is not None else Etiq
   return(resu) 


In [0]:
# Exemple d'utilisation
E=np.array([[20,12,21,12],
            [27,10,9,12],
            [30,15,11,20],
            [32,9,10,10],
            [35,14,11,18]])
P=np.array(["A","B","C","D","E"])
print(Pareto(E))
print(Pareto(E,P))

[[20 12 21 12]
 [27 10  9 12]
 [32  9 10 10]]
[array(['A', 'B', 'D'], dtype='<U1'), array([[20, 12, 21, 12],
       [27, 10,  9, 12],
       [32,  9, 10, 10]])]


In [0]:
""" Algorithme à correction d'étiquettes 
"""

#Fonction qui renvoie le plus court chemin dans l'algorithme à correstion d'etiquette
def PCC(E,source="s",puit="t"):
  chemin=[puit]
  #etiquette du puit correspondant qui minimise le cout
  cout_mini_chemin=E[puit][1][::,0]==min(E[puit][1][::,0])
  pred=E[puit][0][cout_mini_chemin][0]
  chemin.append(pred)
  #remonte les etiquettes du puit jusqu'a la source pour construire le chemin
  while pred!=source:
    cout_mini=E[pred][1][::,0]==min(E[pred][1][::,0])
    pred=E[pred][0][cout_mini][0]
    chemin.append(pred)
  return([chemin[::-1],E[puit][1][cout_mini_chemin]])


def correc_etiquette(g,contraintes,source="s",puit="t"):
  if(source not in g.noeud or puit not in g.noeud):
    print("Erreur: votre source ou votre puit ne figurent pas parmi les sommets de votre graph")
    return(-1)
  #Initialisation
  List=[source]
  Etiq={noeud: [] for noeud in g.noeud}
  Etiq[source]=[np.array([source]),np.array([np.zeros(g.nb_ressources+1)])]
  while(len(List)!=0):
    i=np.random.choice(List)
    List.remove(i)
    if(i in g.arc.keys()):
      for j in g.successeurs(i):
        for k in range(len(Etiq[i][1])):
          E=Etiq[i][1][k]
          if(np.all(g.arc[i][j][1:len(g.arc[i][j])]+E[1:len(E)]<=contraintes)):
            E2=g.arc[i][j]+E
            if len(Etiq[j])==0:
              Etiq[j]=[np.array([i]),np.array([E2])]
            else:
              Etiq[j][1]=np.append(Etiq[j][1],np.array([E2]),axis=0)
              Etiq[j][0]=np.append(Etiq[j][0],i)
            Etiq[j]=Pareto(Etiq[j][1],Etiq[j][0])
            if(E2 in Etiq[j][1]):
              List.append(j)
  return(PCC(Etiq,source=source,puit=puit))


#### **Exemple de résolution**

Un jeune politicien, Paul Politics, a pour objectif de se présenter aux prochaines élections. Sauf surprise, il est assuré de gagner. Cependant, il sait également que la vie d'un politicien classique se termine lorsqu'il finit "déchu", n'ayant plus le soutien de la population. 
Entre le début de la campagne et sa certaine déchéance, son objectif est de minimiser l'argent investi tout en ayant des contraintes sur 3 ressources : le nombre de ses collaborateurs, son énergie dépensée (son niveau d'implication), la distance parcourue en déplacement.
Entre le debut de sa campagne et sa decheance, il doit passer par certaines étapes qui seront les sommets de notre graphe : la source sera donc le debut de la campagne et le puit la decheance.



In [0]:
Paul = Graphe()
Paul.ajout_ressource(["nb_colab","distance","energie"])
Paul.ajoutNoeud(["Debut","Meetings","Conferences","Financement","Corruption","Demagogie","Election","Decheance"])
Paul.ajoutArc(["Debut","Debut","Debut","Financement","Conferences","Conferences","Meetings","Corruption","Corruption","Demagogie","Election"],
              ["Meetings","Conferences","Financement","Conferences","Corruption","Demagogie","Corruption","Election","Decheance","Election","Decheance"],
              [[15,1,3,4],[10,1,4,5],[8,2,6,5],[4,5,6,2],[20,2,0,1],[2,3,9,1],[10,3,0,4],[1,6,2,8],[2,6,6,4],[4,3,2,2],[4,5,6,4]])

"""Vecteur consommation max de ressources 
1 - Pas plus de 12  collaborateurs
2-  Pas plus de 20 unités d'energie
3-  Une distance inférieure à 15 unités"""
conso = np.array([12,20,15])
parcours = correc_etiquette(Paul,conso,"Debut","Decheance")
print(parcours)

[['Debut', 'Meetings', 'Corruption', 'Decheance'], array([[27., 10.,  9., 12.]])]


# Résolution par PuLP du modèle de PLNE

In [0]:
from pulp import *
prob = LpProblem("PLNE", LpMinimize)


Ici, on s'intéressera à la modélisation en PLNE du problème de PCCC (Plus Court Chemin avec Contraintes de ressources). Le problème a été formalisé en première partie du notebook. On cherche le plus court chemin de s à t en respectant les contraintes de ressources.
On considère également les contraintes de ressources finales donc pour chaque contrainte, on aura un majorant sur tout le chemin.

**Variables**  
Pour tout $a \in A$ : $x_{a} \in ${$0,1$}$ = 1 $ si et seulement si $a$ est dans le chemin choisi  

**Objectif**  
Minimiser le coût du chemin:  
$ {min} \sum \limits_{{a \in A}} c_a*x_a $

**Contraintes**  
Conservation du flot:  $ \forall i \ne s,t $ on a  $ \sum \limits_{(i,j)\in \delta^+(i)} x_{ij} = \sum \limits_{(j,i)\in \delta^-(i)} x_{ji}   $

Flot entrant de t:  $ \sum \limits_{(i,t) \in \delta^-(t)} x_{it} = 1 $  
Flot sortant de s:  $ \sum \limits_{(s,i) \in \delta^+(s)} x_{si} = 1 $  
Contraintes de ressources sur les arcs:
$ \forall r\in ${$1,2,\dots, n_r $}, $\sum \limits_{a \in A} t^r_a*x_a \le b^r$ où $b^r$ est la consommation maximale de la ressource $r$.






In [0]:
graphe = Graphe()
x = LpVariable("x",(graphe.nb_arcs(),graphe.nb_arcs()),0,1,LpInteger)